In [1]:
import pandas as pd
import numpy as np
columns=['site_name', 'posa_continent','is_mobile','is_package','channel','srch_ci','srch_co','srch_destination_id', 
'srch_destination_type_id','is_booking','hotel_continent','hotel_market','hotel_cluster','day', 'dayofweek', 'month', 'quarter']
sort=['user_location_country','srch_destination_id', 'srch_destination_type_id','hotel_market','quarter']
train = pd.read_csv("./kaggle/expedia/data/train.csv", chunksize=1000000)
aggs = []
print('-'*38)
for chunk in train:
    props = {}
    chunk['date_time'] = pd.to_datetime(chunk['date_time'])
    for prop in ["month", "day", "dayofweek", "quarter"]:
        props[prop] = getattr(chunk["date_time"].dt, prop)
    chunk.drop('date_time',inplace=True,axis=1)
    ret = pd.DataFrame(props)
    chunk=pd.concat([chunk,ret],axis=1)
    agg = chunk.groupby(columns)['is_booking'].agg(['sum','count'])
    agg.reset_index(inplace=True)
    aggs.append(agg)
    print('.',end='')
print('')
"""
'site_name', 
'posa_continent',
'user_location_country',
'user_location_region', 
'user_location_city',
'orig_destination_distance', 
'user_id', 
'is_mobile', 
'is_package',
'channel', 
'srch_ci', 
'srch_co', 
'srch_adults_cnt', 
'srch_children_cnt',
'srch_rm_cnt', 
'srch_destination_id', 
'srch_destination_type_id',
'is_booking', 
'cnt', 
'hotel_continent', 
'hotel_country', 
'hotel_market',
'hotel_cluster', 
'day', 
'dayofweek', 
'month', 
'quarter'
"""
aggs = pd.concat(aggs, axis=0)
aggs.head(40)

--------------------------------------
......................................


,site_name,posa_continent,is_mobile,is_package,channel,srch_ci,srch_co,srch_destination_id,srch_destination_type_id,is_booking,hotel_continent,hotel_market,hotel_cluster,day,dayofweek,month,quarter,sum,count
0,2,3,0,0,0,2013-01-07,2013-01-09,1536,1,0,6,1414,67,7,0,1,1,0,1
1,2,3,0,0,0,2013-01-07,2013-01-09,1536,1,1,6,1414,67,7,0,1,1,1,1
2,2,3,0,0,0,2013-01-09,2013-01-10,17494,1,0,2,425,18,8,1,1,1,0,1
3,2,3,0,0,0,2013-01-09,2013-01-10,25372,6,0,2,1300,48,7,0,1,1,0,1
4,2,3,0,0,0,2013-01-09,2013-01-10,25372,6,1,2,1300,48,7,0,1,1,1,1
5,2,3,0,0,0,2013-01-09,2013-01-13,12190,6,0,2,637,41,7,0,1,1,0,1
6,2,3,0,0,0,2013-01-09,2013-01-13,12191,6,0,2,637,72,7,0,1,1,0,1
7,2,3,0,0,0,2013-01-09,2013-01-13,12191,6,0,2,637,95,7,0,1,1,0,1
8,2,3,0,0,0,2013-01-10,2013-01-11,1695,1,0,2,1650,14,10,3,1,1,0,1
9,2,3,0,0,0,2013-01-10,2013-01-11,6574,6,0,3,61,17,9,2,1,1,0,1


In [ ]:
CLICK_WEIGHT = 0.05
agg = aggs.groupby(columns).sum().reset_index()
agg['count'] -= agg['sum']
agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
agg['relevance'] = agg['bookings'] + CLICK_WEIGHT * agg['clicks']
agg.head()

In [100]:
def most_popular(group, n_max=5):
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

In [106]:
most_pop = agg.groupby(sort).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

hotel_cluster
user_location_country srch_destination_id srch_destination_type_id hotel_market quarter              
0                     4                   1                        246          2                  81
                                                                                3                  78
                                                                                4                  30
                      8                   1                        416          4            76 39 14
                      9                   3                        2101         1               20 61

In [107]:
test = pd.read_csv('./kaggle/expedia/data/test.csv')
props = {}
test['date_time'] = pd.to_datetime(test['date_time'])
for prop in ["month", "day", "dayofweek", "quarter"]:
    props[prop] = getattr(test["date_time"].dt, prop)
test.drop('date_time',inplace=True,axis=1)
ret = pd.DataFrame(props)
test=pd.concat([test,ret],axis=1)

In [108]:
sort=['user_location_country','srch_destination_id', 'srch_destination_type_id','hotel_market','quarter']
test = test.merge(most_pop, how='left',left_on=sort,right_index=True)
test.head()

,id,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market,day,dayofweek,month,quarter,hotel_cluster
0,0,2,3,66,174,37449,5539.0567,1,1,0,...,12243,6,6,204,27,3,3,9,3,5 37 55 22 68
1,1,2,3,66,174,37449,5873.2923,1,1,0,...,14474,7,6,204,1540,24,3,9,3,NaN
2,2,2,3,66,142,17440,3975.9776,20,0,0,...,11353,1,2,50,699,7,6,6,2,0 31 96 25 59
3,3,2,3,66,258,34156,1508.5975,28,0,1,...,8250,1,2,50,628,14,0,9,3,1 79 45 54 24
4,4,2,3,66,467,36345,66.7913,50,0,0,...,11812,1,2,50,538,17,4,7,3,91 72 48 9 77


In [55]:
preds=test.hotel_cluster

In [57]:
test.drop('hotel_cluster',axis=1,inplace=True)
columns=columns[:-1]

In [94]:
columns=columns[:-1]
#need to collapse the aggs data
#agg=agg.groupby(columns[:-1]).sum().reset_index()
agg.head(19)
most_pop = agg.groupby(columns).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

hotel_cluster
user_location_country srch_destination_id srch_destination_type_id hotel_market hotel_cluster              
0                     4                   1                        246          30                       30
                                                                                78                       78
                                                                                81                       81
                      8                   1                        416          14                       14
                                                                                39                       39

In [87]:
agg.drop(['bookings','clicks','quarter'],axis=1,inplace=True)

In [95]:
most_pop

hotel_cluster
user_location_country srch_destination_id srch_destination_type_id hotel_market hotel_cluster              
0                     4                   1                        246          30                       30
                                                                                78                       78
                                                                                81                       81
                      8                   1                        416          14                       14
                                                                                39                       39
                                                                                76                       76
                      9                   3                        2101         20                       20
                                                                                40                       40
                                                                                60                       60
                                                                                61                       61
                      14                  3                        1434         20                       20
                                                                                38                       38
                                                                                75                       75
                      19                  1                        1522         6                         6
                                                                                38                       38
                                                                                64                       64
                                                                                75                       75
                      21                  1                        1454         30                       30
                                                                                36                       36
                                                                                58                       58
                                                                                78                       78
                                                                                82                       82
                                                                                85                       85
                                                                                90                       90
                      25                  1                        945          33                       33
                      27                  1                        1554         3                         3
                                                                                5                         5
                                                                                20                       20
                                                                                30                       30
                                                                                38                       38
...                                                                                                     ...
239                   54130               6                        1935         36                       36
                                                                                44                       44
                                                                                61                       61
                                                                                82                       82
                                                                                85                

In [96]:

agg[agg.hotel_market==1033]

,user_location_country,srch_destination_id,srch_destination_type_id,hotel_market,hotel_cluster,relevance
208,0,268,1,1033,3,5.35
209,0,268,1,1033,12,1.25
210,0,268,1,1033,20,4.25
211,0,268,1,1033,25,1.20
212,0,268,1,1033,29,0.35
213,0,268,1,1033,35,0.05
214,0,268,1,1033,36,2.45
215,0,268,1,1033,38,3.65
216,0,268,1,1033,44,0.15
217,0,268,1,1033,46,9.30


In [97]:
train.head(20)

AttributeError: 'TextFileReader' object has no attribute 'head'

In [23]:
import pandas as pd
import numpy as np
chunk = pd.read_csv("./kaggle/expedia/data/train.csv",
              
                    nrows=10000)

In [8]:
train["date_time"] = pd.to_datetime(train["date_time"])

In [33]:
import pandas as pd
import numpy as np
chunk = pd.read_csv("./kaggle/expedia/data/train.csv", nrows=10000)
chunk['date_time'] = pd.to_datetime(chunk['date_time'])
for prop in ["month", "day", "dayofweek", "quarter"]:
    props[prop] = getattr(chunk["date_time"].dt, prop)
chunk.drop('date_time',inplace=True,axis=1)
ret = pd.DataFrame(props)
train=pd.concat([chunk,ret],axis=1)

In [15]:
agg.sort(columns=['user_location_country','srch_destination_id','srch_destination_type_id','hotel_market','hotel_cluster','quarter','relevance'])

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,user_location_country,srch_destination_id,srch_destination_type_id,hotel_market,hotel_cluster,quarter,bookings,clicks,relevance
0,0,4,1,246,30,4,1,1,1.05
1,0,4,1,246,78,3,1,2,1.10
2,0,4,1,246,81,2,1,1,1.05
3,0,8,1,416,14,4,0,1,0.05
4,0,8,1,416,39,4,0,1,0.05
5,0,8,1,416,76,4,0,1,0.05
6,0,9,3,2101,20,1,0,5,0.25
7,0,9,3,2101,40,2,1,3,1.15
8,0,9,3,2101,60,2,1,2,1.10
9,0,9,3,2101,61,1,0,1,0.05


In [17]:
columns=['user_location_country','srch_destination_id','srch_destination_type_id','hotel_market','hotel_cluster','quarter','relevance']

In [8]:
relevance = agg['relevance'].values
hotel_cluster = agg['hotel_cluster'].values
most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:5]
np.array_str(most_popular)[1:-1] # remove square brackets
#hotel_cluster[np.argsort(relevance)[::-1]][:5]

' 1  1  1  1 79'

In [10]:
hotel_cluster[np.argsort(relevance)[::-1]]

array([ 1,  1,  1, ..., 30,  7, 20], dtype=int64)

In [11]:
def most_popular(group, columns, n_max=5):
    relevance = group['relevance'].values
    cluster = group[columns].values
    most_popular = hotel_cluster[np.argsort([columns,relevance])[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

In [19]:
columns=['user_location_country','srch_destination_id', 'srch_destination_type_id', 
                        'hotel_market','quarter']


In [16]:
aggs.drop

,user_location_country,srch_destination_id,srch_destination_type_id,hotel_market,hotel_cluster,quarter,sum,count
0,0,104,3,1483,82,4,0,1
1,0,137,3,1605,53,3,0,1
2,0,137,3,1605,61,3,1,2
3,0,137,3,1605,62,3,0,1
4,0,137,3,1605,81,3,0,1
5,0,137,3,1605,82,4,2,4
6,0,317,3,1549,36,2,1,2
7,0,317,3,1549,36,3,1,2
8,0,347,1,52,20,4,0,1
9,0,468,1,153,2,3,1,4


In [52]:
test

,id,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market,day,dayofweek,month,quarter
0,0,2,3,66,174,37449,5539.0567,1,1,0,...,1,12243,6,6,204,27,3,3,9,3
1,1,2,3,66,174,37449,5873.2923,1,1,0,...,1,14474,7,6,204,1540,24,3,9,3
2,2,2,3,66,142,17440,3975.9776,20,0,0,...,1,11353,1,2,50,699,7,6,6,2
3,3,2,3,66,258,34156,1508.5975,28,0,1,...,1,8250,1,2,50,628,14,0,9,3
4,4,2,3,66,467,36345,66.7913,50,0,0,...,1,11812,1,2,50,538,17,4,7,3
5,5,2,3,66,311,48189,359.8521,51,0,0,...,2,11827,1,2,50,447,21,1,7,3
6,6,2,3,66,311,48189,237.3465,51,0,0,...,1,8271,1,2,50,696,29,2,7,3
7,7,2,3,66,348,24811,216.5785,51,0,0,...,1,8291,1,2,50,191,1,5,8,3
8,8,2,3,66,311,48189,2337.6754,51,0,0,...,1,8250,1,2,50,628,7,5,11,4
9,9,2,3,66,311,48189,2539.7995,51,0,0,...,1,9145,1,2,50,364,8,6,11,4


In [104]:
test

,id,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market,day,dayofweek,month,quarter
0,0,2,3,66,174,37449,5539.0567,1,1,0,...,1,12243,6,6,204,27,3,3,9,3
1,1,2,3,66,174,37449,5873.2923,1,1,0,...,1,14474,7,6,204,1540,24,3,9,3
2,2,2,3,66,142,17440,3975.9776,20,0,0,...,1,11353,1,2,50,699,7,6,6,2
3,3,2,3,66,258,34156,1508.5975,28,0,1,...,1,8250,1,2,50,628,14,0,9,3
4,4,2,3,66,467,36345,66.7913,50,0,0,...,1,11812,1,2,50,538,17,4,7,3
5,5,2,3,66,311,48189,359.8521,51,0,0,...,2,11827,1,2,50,447,21,1,7,3
6,6,2,3,66,311,48189,237.3465,51,0,0,...,1,8271,1,2,50,696,29,2,7,3
7,7,2,3,66,348,24811,216.5785,51,0,0,...,1,8291,1,2,50,191,1,5,8,3
8,8,2,3,66,311,48189,2337.6754,51,0,0,...,1,8250,1,2,50,628,7,5,11,4
9,9,2,3,66,311,48189,2539.7995,51,0,0,...,1,9145,1,2,50,364,8,6,11,4
